# stockdata

> This module does the following: 1) Pull latest data from NSE 2) Calculate various technical indicators 3) Save the updated, generated data.

In [ ]:
#| default_exp stockdata

In [ ]:
#| export
def say_hello(to:str):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
say_hello("Ankur")

'Hello Ankur!'

In [ ]:
assert say_hello("Hamel")=="Hello Hamel!"

In [ ]:
from fastcore.test import *

In [ ]:
test_eq(say_hello("Ankur"), "Hello Ankur!")

AssertionError: ==:
Hello Hamel!
Hello Ankur!

In [ ]:
from IPython.display import display,SVG

In [ ]:
display(SVG('<svg height="100" xmlns="http://www.w3.org/2000/svg"><circle cx="50" cy="50" r="40"/></svg>'))

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def foo(): pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()